In [1]:
import pandas as pd
import geopandas as gpd
import shapely
from shapely.geometry import Point, LineString, MultiLineString
from tqdm.notebook import tqdm
import numpy as np

In [5]:
df_dupl = pd.read_csv('intervs_st.csv',sep=';',encoding='windows-1251')

In [6]:
df = df_dupl.drop_duplicates(subset=['line_id']).reset_index(drop=True)

In [7]:
df = df.sort_values(by=['населенный_пункт',
                        'тип_тс','номер_маршрута']).reset_index(drop=True)
df['line_id'] = df['line_id'].astype(str)

In [8]:
df.to_csv('rts_ya.csv',sep=';',encoding='utf-8-sig')

In [10]:
gdf_lines_dupl = gpd.read_file('./rts_ya/gdf_line_all_n_samobl.json',encoding='utf-8')

In [11]:
gdf_lines = gdf_lines_dupl.drop_duplicates(subset=['line_id']).reset_index(drop=True)

In [19]:
full_geo_interv = df.merge(gdf_lines[['line_id','geometry']],how='left',on=['line_id'])

In [20]:
full_geo_interv = gpd.GeoDataFrame(full_geo_interv)
full_geo_interv.crs='epsg:4326'

In [37]:
full_geo_interv.head(2)

,line_id,населенный_пункт,тип_тс,тип_маршрута,номер_маршрута,наим_маршрута,интерв_мин_буд,частота,geometry
0,2193178991,Самара,автобус,муниципальный,1,"а/ст ""Красная Глинка"" - Ж/д вокзал (Поликлиника)",8,7.5,"LINESTRING (50.12249 53.18858, 50.12277 53.188..."
1,2193178998,Самара,автобус,пригородный,101а,"г. Самара (а/ст ""Аврора"") - с. Лопатино",30,2.0,"LINESTRING (50.19138 53.19107, 50.19141 53.190..."


In [28]:
def get_grp_seg(full_geo_interv):

    np_fgi = full_geo_interv.to_numpy()
    ind_geo = list(full_geo_interv.columns).index('geometry')
    ind_tpts = list(full_geo_interv.columns).index('тип_тс')
    ind_freq = list(full_geo_interv.columns).index('частота')
    ind_lnid = list(full_geo_interv.columns).index('line_id')
    ind_rtnmbr = list(full_geo_interv.columns).index('номер_маршрута')


    bg_lst_segm = []
    bg_lst_tpts = []
    bg_lst_freq = []
    bg_lst_wkt = []

    bg_lst_rtnmbr = []
    bg_lst_lineid = []
    i=0
    for i in tqdm(range(len(np_fgi))):
    # for i in range(2):
        one_line = np_fgi[i,ind_geo]
        one_tpts = np_fgi[i,ind_tpts]
        one_freq = np_fgi[i,ind_freq]
        one_lineid = np_fgi[i,ind_lnid]
        one_rtnmbr = np_fgi[i,ind_rtnmbr]

        intrsect = one_line.intersection(one_line)
        lst_segm_new = list(intrsect)
        lst_one_tpts = []
        lst_one_freq = []
        lst_one_wkt = []

        lst_one_lineid = []
        lst_one_rtnmbr = []
        for j in range(len(lst_segm_new)):
            if ((one_tpts == 'троллейбус') | (one_tpts == 'автобус')):
                tp_ts_grp = 'not_rail'
            else:
                tp_ts_grp = 'rail'
            #
            lst_one_tpts.append(tp_ts_grp)
            lst_one_freq.append(one_freq)
            lst_one_wkt.append(lst_segm_new[j].to_wkt())
            lst_one_lineid.append(one_lineid)
            lst_one_rtnmbr.append(one_rtnmbr)
        #
        bg_lst_segm = bg_lst_segm + lst_segm_new
        bg_lst_tpts = bg_lst_tpts + lst_one_tpts
        bg_lst_freq = bg_lst_freq + lst_one_freq
        bg_lst_wkt = bg_lst_wkt + lst_one_wkt
        bg_lst_rtnmbr = bg_lst_rtnmbr + lst_one_rtnmbr
        bg_lst_lineid = bg_lst_lineid + lst_one_lineid
    # 


    df_seg = gpd.GeoDataFrame(data=list(zip(bg_lst_tpts, bg_lst_freq, bg_lst_wkt,
                                            bg_lst_rtnmbr,bg_lst_lineid)), 
                              geometry=bg_lst_segm, 
                              columns=['type_ts','frequency','geo_wkt',
                                       'route_number','line_id'])
    df_seg.crs='epsg:4326'
    #

    grp_seg = pd.DataFrame(df_seg.groupby(['geo_wkt',
                                           'type_ts']).sum()['frequency']).reset_index()

    grp_seg = grp_seg.sort_values(by=['type_ts','frequency'],ascending=False)

    grp_seg = grp_seg.merge(df_seg[['geo_wkt',
                                    'geometry']].drop_duplicates(subset=['geo_wkt']), 
                            how='left', on=['geo_wkt']).reset_index(drop=True)
    #
    grp_seg = gpd.GeoDataFrame(grp_seg)
    grp_seg.crs='epsg:4326'


    np_gs = grp_seg.to_numpy()
    ind_wkt = list(grp_seg.columns).index('geo_wkt')
    ind_tpts = list(grp_seg.columns).index('type_ts')

    np_ds = df_seg.to_numpy()
    ind_wkt2 = list(df_seg.columns).index('geo_wkt')
    ind_tpts2 = list(df_seg.columns).index('type_ts')
    ind_rtnb2 = list(df_seg.columns).index('route_number')
    ind_lnd2 = list(df_seg.columns).index('line_id')

    lst_rts_in_seg = []
    lst_lineid_in_seg = []
    i=0
    for i in tqdm(range(len(np_gs))):
    # for i in range(10):
        one_wkt = np_gs[i,ind_wkt]
        one_tpts = np_gs[i,ind_tpts]
        prt_df = np_ds[np.where((np_ds[:,ind_wkt2] == one_wkt) 
                                * (np_ds[:,ind_tpts2] == one_tpts))]

        lst_lineid = list(set(list(prt_df[:,ind_lnd2])))
        lst_rtnmbr = list(set(list(prt_df[:,ind_rtnb2])))
        lst_rts_in_seg.append(str(lst_rtnmbr).replace('[','').replace(']',''))
        lst_lineid_in_seg.append(str(lst_lineid).replace('[','').replace(']',''))
    #

    grp_seg['rtnmbr_in_seg'] = lst_rts_in_seg
    grp_seg['lineid_in_seg'] = lst_lineid_in_seg

    return grp_seg

In [29]:
sam_rail = full_geo_interv[((full_geo_interv.населенный_пункт == 'Самара') 
                                     & (full_geo_interv.тип_тс == 'трамвай'))].reset_index(drop=True)
grp_seg_sam_rail = get_grp_seg(sam_rail)

In [32]:
grp_seg_sam_rail.to_file('grp_seg_sam_rail.json',driver='GeoJSON',encoding='utf-8')

In [34]:
sam_norail = full_geo_interv[((full_geo_interv.населенный_пункт == 'Самара') 
                                     & (full_geo_interv.тип_тс != 'трамвай'))].reset_index(drop=True)
grp_seg_sam_norail = get_grp_seg(sam_norail)

In [35]:
tol_norail = full_geo_interv[((full_geo_interv.населенный_пункт == 'Тольятти') 
                                     & (full_geo_interv.тип_тс != 'трамвай'))].reset_index(drop=True)
grp_seg_tol_norail = get_grp_seg(tol_norail)

In [36]:
grp_seg_tol_norail.to_file('grp_seg_tol_norail.json',driver='GeoJSON',encoding='utf-8')
grp_seg_sam_norail.to_file('grp_seg_sam_norail.json',driver='GeoJSON',encoding='utf-8')

In [38]:
full_geo_interv.to_file('full_geo_interv.json',driver='GeoJSON',encoding='utf-8')